## Modelling Exercise

<b>Scenario</b>

Offer 1 of 3 different products, namely Mutual Fund (MF), Credit Card (CC), & Consumer Loan (CL) to only 100 clients of a bank, out of a dataset of 1615 clients. Clients can only receive one offer for one product.

<b>Goal</b>

Select best 100 people to offer products which result to maximum potential revenue.

<b>Description of Data</b>

Dataset contains information of the 1615 clients. Included are:
1. Social Demographic Data;
2. Products currently owned and balances;
3. Behavioral data from products (cash flow); and
4. Sales and Revenue from previous marketing campaign. Only 60% (969 clients) have this information, should be used for modelling. Thus, the top 100 people to be chosen are from the remaining 40% (646 clients).


<b>Questions</b>
1.	Which clients are more likely to buy consumer loan? 
2.	Which clients are more likely to buy credit card? 
3.	Which clients are more likely to buy mutual fund? 
4.	Which clients are to be targeted with which offer? General description.  
5.	What would be the expected revenue based on your strategy? 
6.	How well do your models perform?
7.	How did you ensure the models would continue perform well once applied to the 40% of clients?

<b>Approach</b>

To answer questions 1-4, clients who bought each product are separated to those who did not, and the frequency distribution for each feature (from Social Demographic Data to Behavioral Data) are compared. These features will be used to model the sales and revenue of each product.

The 60% dataset will be split into training and testing datasets for cross validation of the models. After fitting the model, the remaining 40% clients will then be offered only one product. For each client, the product with the highest revenue will only be offered.



<b>Logistic Regression</b> is used in predicting whether or not the clients purchase the product. Results are categorical, i.e., 1 if client accepted offer & 0 if declined. LR is easier to visualize and may be more robust than Tree-based classifiers which are prone to overfitting.
 
<b>Linear Regression</b> is used to model the revenue from clients who purchased.

One model each for every product.

<b>Answers to Questions</b>

1. In terms of social demography, clients in their early thirties (specifically 32) with tenure of 151 months are more likely to buy CL;
2. Clients in their 30s (specifically 30) with tenure of 150 months are more likely to buy CC; and
3. Male clients in their late 30s to early 40s (specifically 39) with tenure of 151 months are more likely to buy MF.
4. Clients are targeted with the greatest possible revenue from all possible offers. (ex. if a client is predicted to be offered two products, the product with the higher revenue will be offered)
5. Approximately more than 500 Euros.
6. Models in predicting whether clients buy the product have high scores, whereas predicting revenue obtained lower scores.
7. The 60% dataset was split into training and testing sets for cross validation of model performance. Scores were needed to be consistent as possible on both sets. This way, the models would perform well once applied to the rest.

In [23]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

In [24]:
df = pd.read_excel('Data.xlsx', sheet_name=['Soc_Dem', 'Products_ActBalance', 'Inflow_Outflow', 'Sales_Revenues'])

Soc_Dem = df['Soc_Dem'].sort_values('Client').fillna('M') #(3 clients did not specify sex, they are assumed M)
Prod_ActBal = df['Products_ActBalance'].sort_values('Client')
Cash_Flow = df['Inflow_Outflow'].sort_values('Client').reset_index(drop=True)
Sales_Rev = df['Sales_Revenues'].sort_values('Client').reset_index(drop=True)

Creating training set dataframe

In [25]:
train_df = Soc_Dem.merge(Prod_ActBal, how='right', on='Client').merge(Cash_Flow, how='right', on='Client').merge(Sales_Rev, how='right', on='Client')
train_df.fillna(value=0, inplace=True)
train_df['Sex'] = LabelEncoder().fit_transform(train_df['Sex'].astype(str))
train_df['Sex'] = train_df['Sex'].replace(to_replace=0, value=1)-1
print(train_df.shape)
train_df.head()

(969, 36)


,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,...,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sale_MF,Sale_CC,Sale_CL,Revenue_MF,Revenue_CC,Revenue_CL
0,1,0,51.0,7.0,1.0,0.0,0.0,1.0,0.0,0.0,...,30.0,3.0,12.0,9.0,1,0,0,26.972679,0.0,0.0
1,2,1,43.0,152.0,1.0,1.0,0.0,0.0,0.0,0.0,...,6.0,2.0,2.0,1.0,0,0,0,0.000000,0.0,0.0
2,6,0,24.0,19.0,1.0,0.0,0.0,0.0,0.0,1.0,...,6.0,1.0,0.0,0.0,0,0,0,0.000000,0.0,0.0
3,8,0,64.0,30.0,1.0,0.0,0.0,1.0,0.0,0.0,...,24.0,10.0,1.0,0.0,0,0,0,0.000000,0.0,0.0
4,13,0,66.0,132.0,1.0,1.0,0.0,0.0,0.0,1.0,...,39.0,6.0,17.0,10.0,0,0,0,0.000000,0.0,0.0


Creating test set dataframe (used for prediction and selection of 100 clients)

In [26]:
main_df = Soc_Dem.merge(Prod_ActBal, how='left', on='Client').merge(Cash_Flow, how='left', on='Client')
main_df.fillna(value=0, inplace=True)
main_df['Sex'] = LabelEncoder().fit_transform(main_df['Sex'].astype(str))
main_df['Sex'] = main_df['Sex'].replace(to_replace=0, value=1)-1

cond = main_df['Client'].isin(train_df['Client'])
toPredict_df = main_df.drop(main_df[cond].index).reset_index(drop=True)
print(toPredict_df.shape)
toPredict_df.head()

(646, 30)


,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,...,VolumeDeb,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder
0,3,0,17,140,1,0.0,1.0,0.0,0.0,0.0,...,123.750000,123.750000,0.000000,112.857143,10.678571,3.0,3.0,0.0,1.0,1.0
1,4,0,24,153,1,1.0,0.0,0.0,1.0,0.0,...,3614.747500,3598.950357,714.285714,526.037500,1963.165357,41.0,36.0,6.0,12.0,13.0
2,5,0,58,200,1,1.0,0.0,0.0,0.0,0.0,...,5868.460714,4031.925000,157.142857,832.175000,1779.571429,44.0,41.0,4.0,17.0,13.0
3,7,0,55,227,1,0.0,0.0,0.0,0.0,0.0,...,99.043929,99.043929,17.857143,69.151071,10.714286,10.0,10.0,1.0,6.0,1.0
4,9,0,33,182,1,0.0,0.0,1.0,0.0,0.0,...,415.460714,375.705357,53.571429,0.000000,170.714286,10.0,7.0,1.0,0.0,3.0


### Creating functions to train model and return its training and testing scores

In [27]:
def sales_train_model(features, target, size, state):
    x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=size, random_state=state)
    model = LogisticRegression(max_iter=10000000000000000000, random_state=state)   #max_iters have to be specified
    model_trained = model.fit(x_train, y_train)

    train_accuracy = model_trained.score(x_train, y_train)
    test_accuracy = model_trained.score(x_test, y_test)
    y_predicted = model.predict(x_test)

    print("Training accuracy:                  " + str(train_accuracy))
    print("Cross-validation training accuracy: " + str(test_accuracy) + "\n")
    print(classification_report(y_test, y_predicted))   # to also check precision, recall, etc.

    return model_trained

def revenue_train_model(features, target, size, state):
    x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=size, random_state=state)
    model = LinearRegression()
    model_trained = model.fit(x_train, y_train)

    train_accuracy = model_trained.score(x_train, y_train)
    test_accuracy = model_trained.score(x_test, y_test)

    print("Training accuracy:                  " + str(train_accuracy))
    print("Cross-validation test accuracy:    " + str(test_accuracy))

    return model_trained

def best_features(features, target, model, n_feats):
    importantFeats = RFE(model, n_features_to_select=n_feats)
    importantFeats = importantFeats.fit(features, target)

    return importantFeats.support_

## Modelling Sales Offer and Revenue

Sales is predicted by logistic regression using specific features found in Soc_Dem, Products_ActBalance, and Inflow_Outflow. Revenue is computed by linear regression using similar features in sales, only when the clients (rows) have bought the products, i.e., Sales_CC, CL, or MF == 1.

#### MF Sales and Revenue

Features [1,2,3,6,8,11,12,14,15,17,19,20,21,22,25,26,28] were found to be important from data visualization. This is disregarded initially, and all features are initally used for modelling.

Feature values are scaled using StandardScaler() due to having different ranges of values. This helps alleviate biases/skewness.

In [28]:
test_partition = 0.5
rand_state = None       # specify for testing, can be removed
sc = StandardScaler()

Training set is partitioned equally so that the training and testing size will either not be too big or too small, especially since it should model the whole 1615 dataset.

In [29]:
Cols_MF = np.arange(1,30) # to specify important features, also assumed that revenue features are similar

features_saleMF = sc.fit_transform(train_df.iloc[:,Cols_MF])
target_saleMF = train_df['Sale_MF']

revenueMF = train_df[target_saleMF==1]
features_revenueMF = sc.fit_transform(revenueMF.iloc[:,Cols_MF])
target_revenueMF = revenueMF['Revenue_MF']

model_saleMF = sales_train_model(features_saleMF, target_saleMF, test_partition, rand_state)
model_revenueMF = revenue_train_model(features_revenueMF, target_revenueMF, test_partition, rand_state)

Training accuracy:                  0.8037190082644629
Cross-validation training accuracy: 0.822680412371134

              precision    recall  f1-score   support

           0       0.83      0.98      0.90       396
           1       0.59      0.11      0.19        89

    accuracy                           0.82       485
   macro avg       0.71      0.55      0.54       485
weighted avg       0.79      0.82      0.77       485

Training accuracy:                  0.4363097926387207
Cross-validation test accuracy:    -13.645709646406742


Selecting 14 best features according to RFE plus additional features related to the product (in terms of feature names):

(We choose 14 since there are 29 features in total, and aim to reduce features in half)

In [30]:
n_feats = 14
best_features(features_saleMF, target_saleMF, model_saleMF, n_feats)

array([False, False, False, False, False,  True, False, False, False,
        True,  True, False, False, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True, False,  True, False,
        True, False])

New features according to RFE: [6,10,11,15,16,17,18,20,21,22,23,24,26,28]

In [31]:
Cols_MF = [6,10,11,15,16,17,18,20,21,22,23,24,26,28]

features_saleMF = sc.fit_transform(train_df.iloc[:,Cols_MF])
target_saleMF = train_df['Sale_MF']

revenueMF = train_df[target_saleMF==1]
features_revenueMF = sc.fit_transform(revenueMF.iloc[:,Cols_MF])
target_revenueMF = revenueMF['Revenue_MF']

model_saleMF = sales_train_model(features_saleMF, target_saleMF, test_partition, rand_state)
model_revenueMF = revenue_train_model(features_revenueMF, target_revenueMF, test_partition, rand_state)

Training accuracy:                  0.8016528925619835
Cross-validation training accuracy: 0.8268041237113402

              precision    recall  f1-score   support

           0       0.84      0.98      0.90       393
           1       0.67      0.17      0.28        92

    accuracy                           0.83       485
   macro avg       0.75      0.58      0.59       485
weighted avg       0.80      0.83      0.78       485

Training accuracy:                  0.37094312555911124
Cross-validation test accuracy:    -2.6886250793517807


#### CC Sales and Revenue

Useful features according to data visualization and coefs_ : [2,3,6,11,12,13,14,15,16,17,18,19,27,28]

In [32]:
Cols_CC = np.arange(1,30)

features_saleCC = sc.fit_transform(train_df.iloc[:,Cols_CC])
target_saleCC = train_df['Sale_CC']

revenueCC = train_df[target_saleCC==1]
features_revenueCC = sc.fit_transform(revenueCC.iloc[:,Cols_CC])
target_revenueCC = revenueCC['Revenue_CC']

model_saleCC = sales_train_model(features_saleCC, target_saleCC, test_partition, rand_state)
model_revenueCC = revenue_train_model(features_revenueCC, target_revenueCC, test_partition, rand_state)

Training accuracy:                  0.8037190082644629
Cross-validation training accuracy: 0.7546391752577319

              precision    recall  f1-score   support

           0       0.77      0.96      0.85       363
           1       0.55      0.14      0.22       122

    accuracy                           0.75       485
   macro avg       0.66      0.55      0.54       485
weighted avg       0.71      0.75      0.70       485

Training accuracy:                  0.15605473000067172
Cross-validation test accuracy:    -18.21619418933251


Selecting 14 best features + features related to product:

In [33]:
best_features(features_saleCC, target_saleCC, model_saleCC, n_feats)

array([False, False, False, False, False, False, False,  True,  True,
        True,  True, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True, False,  True, False, False,  True,
        True, False])

New features: [8,9,10,11,16,17,18,19,20,21,22,24,27,28]

In [34]:
Cols_CC = [8,9,10,11,16,17,18,19,20,21,22,24,27,28]

features_saleCC = sc.fit_transform(train_df.iloc[:,Cols_CC])
target_saleCC = train_df['Sale_CC']

revenueCC = train_df[(target_saleCC == 1)]
features_revenueCC = sc.fit_transform(revenueCC.iloc[:,Cols_CC])
target_revenueCC = revenueCC['Revenue_CC']

model_saleCC = sales_train_model(features_saleCC, target_saleCC, test_partition, rand_state)
model_revenueCC = revenue_train_model(features_revenueCC, target_revenueCC, test_partition, rand_state)

Training accuracy:                  0.7830578512396694
Cross-validation training accuracy: 0.7876288659793814

              precision    recall  f1-score   support

           0       0.80      0.97      0.88       375
           1       0.62      0.16      0.26       110

    accuracy                           0.79       485
   macro avg       0.71      0.57      0.57       485
weighted avg       0.76      0.79      0.74       485

Training accuracy:                  0.1666061654724993
Cross-validation test accuracy:    -0.3625403010820647


#### CL Sales and Revenue

Useful features according to data visualization and coefs_ : [2,3,4,5,10,11,13,16,17,20,21,26,27,28,29]

In [35]:
Cols_CL = np.arange(1,30)

features_saleCL = sc.fit_transform(train_df.iloc[:,Cols_CL])
target_saleCL = train_df['Sale_CL']

revenueCL = train_df[target_saleCL==1]
features_revenueCL = sc.fit_transform(revenueCL.iloc[:,Cols_CL])
target_revenueCL = revenueCL['Revenue_CL']

model_saleCL = sales_train_model(features_saleCL, target_saleCL, test_partition, rand_state)
model_revenueCL = revenue_train_model(features_revenueCL, target_revenueCL, test_partition, rand_state)

Training accuracy:                  0.7913223140495868
Cross-validation training accuracy: 0.6989690721649484

              precision    recall  f1-score   support

           0       0.72      0.92      0.81       337
           1       0.52      0.20      0.28       148

    accuracy                           0.70       485
   macro avg       0.62      0.56      0.55       485
weighted avg       0.66      0.70      0.65       485

Training accuracy:                  0.6379648160938804
Cross-validation test accuracy:    -24.946340263346006


Selecting 14 best features + features related to product:

In [36]:
best_features(features_saleCL, target_saleCL, model_saleCL, n_feats)

array([False,  True,  True,  True,  True, False, False, False, False,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False,  True, False, False, False, False, False, False,  True,
       False, False])

New features: [2,3,4,5,10,11,12,13,14,16,17,18,20,27]

In [37]:
Cols_CL = [2,3,4,5,10,11,12,13,14,16,17,18,20,27]

features_saleCL = sc.fit_transform(train_df.iloc[:,Cols_CL])
target_saleCL = train_df['Sale_CL']

revenueCL = train_df[train_df['Sale_CL']==1]
features_revenueCL = sc.fit_transform(revenueCL.iloc[:,Cols_CL])
target_revenueCL = revenueCL['Revenue_CL']

model_saleCL = sales_train_model(features_saleCL, target_saleCL, test_partition, rand_state)
model_revenueCL = revenue_train_model(features_revenueCL, target_revenueCL, test_partition, rand_state)

Training accuracy:                  0.7644628099173554
Cross-validation training accuracy: 0.7298969072164948

              precision    recall  f1-score   support

           0       0.74      0.94      0.83       335
           1       0.66      0.26      0.37       150

    accuracy                           0.73       485
   macro avg       0.70      0.60      0.60       485
weighted avg       0.72      0.73      0.69       485

Training accuracy:                  0.06550675285904217
Cross-validation test accuracy:    -0.09288198003419956


## Predicting if remaining clients will buy products

In [38]:
def get_Prediction(dfToPredict, features, model):
    clientFeatures_toPredict = sc.fit_transform(dfToPredict.iloc[:,features])
    prediction = model.predict(clientFeatures_toPredict)
    return prediction

CC Sales and Revenue

In [39]:
saleMF_prediction = get_Prediction(toPredict_df, Cols_MF, model_saleMF)
saleCC_prediction = get_Prediction(toPredict_df, Cols_CC, model_saleCC)
saleCL_prediction = get_Prediction(toPredict_df, Cols_CL, model_saleCL)

toPredict_df['Sale_MF'] = saleMF_prediction
toPredict_df['Sale_CC'] = saleCC_prediction
toPredict_df['Sale_CL'] = saleCL_prediction

toPredict_df

,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,...,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sale_MF,Sale_CC,Sale_CL
0,3,0,17,140,1,0.0,1.0,0.0,0.0,0.0,...,112.857143,10.678571,3.0,3.0,0.0,1.0,1.0,0,0,0
1,4,0,24,153,1,1.0,0.0,0.0,1.0,0.0,...,526.037500,1963.165357,41.0,36.0,6.0,12.0,13.0,0,0,0
2,5,0,58,200,1,1.0,0.0,0.0,0.0,0.0,...,832.175000,1779.571429,44.0,41.0,4.0,17.0,13.0,0,1,1
3,7,0,55,227,1,0.0,0.0,0.0,0.0,0.0,...,69.151071,10.714286,10.0,10.0,1.0,6.0,1.0,0,0,0
4,9,0,33,182,1,0.0,0.0,1.0,0.0,0.0,...,0.000000,170.714286,10.0,7.0,1.0,0.0,3.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,1606,0,47,78,1,0.0,0.0,0.0,0.0,1.0,...,129.318214,219.642857,19.0,17.0,0.0,7.0,5.0,0,0,0
642,1609,0,31,111,1,0.0,0.0,0.0,0.0,0.0,...,0.000000,92.928571,2.0,2.0,0.0,0.0,2.0,0,0,0
643,1610,0,48,92,1,0.0,0.0,1.0,0.0,0.0,...,43.889286,169.321429,20.0,18.0,3.0,2.0,7.0,0,0,0
644,1611,0,41,181,1,0.0,0.0,1.0,0.0,0.0,...,32.142857,890.578571,11.0,11.0,1.0,3.0,6.0,0,0,0


Predicting revenue for clients who will be offered

In [40]:
MF_clients = toPredict_df[toPredict_df['Sale_MF'] == 1]
CC_clients = toPredict_df[toPredict_df['Sale_CC'] == 1]
CL_clients = toPredict_df[toPredict_df['Sale_CL'] == 1]

revenueMF_prediction = get_Prediction(MF_clients, Cols_MF, model_revenueMF)
revenueCC_prediction = get_Prediction(CC_clients, Cols_CC, model_revenueCC)
revenueCL_prediction = get_Prediction(CL_clients, Cols_CL, model_revenueCL)

# create tentatively empty columns
toPredict_df[['Revenue_MF', 'Revenue_CC', 'Revenue_CL']] = 0

toPredict_df.loc[toPredict_df['Sale_MF'] == 1, 'Revenue_MF'] = revenueMF_prediction
toPredict_df.loc[toPredict_df['Sale_CC'] == 1, 'Revenue_CC'] = revenueCC_prediction
toPredict_df.loc[toPredict_df['Sale_CL'] == 1, 'Revenue_CL'] = revenueCL_prediction

toPredict_df[(toPredict_df['Sale_CL'] == 1) | (toPredict_df['Sale_MF'] == 1) | (toPredict_df['Sale_CC'] == 1)]

,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,...,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sale_MF,Sale_CC,Sale_CL,Revenue_MF,Revenue_CC,Revenue_CL
2,5,0,58,200,1,1.0,0.0,0.0,0.0,0.0,...,41.0,4.0,17.0,13.0,0,1,1,0.000000,6.235377,11.787074
10,19,0,46,94,1,1.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,4.0,0,1,0,0.000000,5.018632,0.000000
14,30,0,54,2,1,1.0,2.0,1.0,1.0,0.0,...,19.0,4.0,26.0,9.0,1,0,0,-15.145685,0.000000,0.000000
17,41,0,20,198,1,0.0,0.0,1.0,0.0,0.0,...,18.0,6.0,7.0,1.0,0,0,1,0.000000,0.000000,12.655955
21,57,0,28,157,1,0.0,0.0,0.0,0.0,0.0,...,6.0,1.0,2.0,1.0,0,0,1,0.000000,0.000000,11.327915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,1516,0,40,192,1,0.0,0.0,0.0,1.0,0.0,...,13.0,2.0,23.0,10.0,1,0,1,-1.407454,0.000000,12.405535
629,1569,0,3,152,1,3.0,32.0,0.0,0.0,0.0,...,20.0,0.0,13.0,6.0,1,0,0,11.143192,0.000000,0.000000
636,1597,0,19,181,1,0.0,0.0,1.0,0.0,0.0,...,17.0,3.0,6.0,4.0,0,0,1,0.000000,0.000000,14.286525
638,1599,0,25,220,1,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,0.0,0.0,0,0,1,0.000000,0.000000,16.119829


Observation: occassionally there are less than 100 clients predicted to purchase. This may be due to underfitting of the model, which may be prevented by increasing features used.

Creating column 'Product_to_Offer' which describes which product is best based on revenue and another column 'Product Revenue' which shows the revenue from the offered product.

In [41]:
toPredict_df['Product_to_offer'] = toPredict_df[['Revenue_MF', 'Revenue_CC', 'Revenue_CL']].idxmax(axis=1)
toPredict_df['Max_Revenue'] = toPredict_df[['Revenue_MF', 'Revenue_CC', 'Revenue_CL']].max(axis=1)

top_clients = toPredict_df[(toPredict_df['Sale_CL'] == 1) | (toPredict_df['Sale_MF'] == 1) | (toPredict_df['Sale_CC'] == 1)]
top_clients

,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,...,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sale_MF,Sale_CC,Sale_CL,Revenue_MF,Revenue_CC,Revenue_CL,Product_to_offer,Max_Revenue
2,5,0,58,200,1,1.0,0.0,0.0,0.0,0.0,...,17.0,13.0,0,1,1,0.000000,6.235377,11.787074,Revenue_CL,11.787074
10,19,0,46,94,1,1.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0,1,0,0.000000,5.018632,0.000000,Revenue_CC,5.018632
14,30,0,54,2,1,1.0,2.0,1.0,1.0,0.0,...,26.0,9.0,1,0,0,-15.145685,0.000000,0.000000,Revenue_CC,0.000000
17,41,0,20,198,1,0.0,0.0,1.0,0.0,0.0,...,7.0,1.0,0,0,1,0.000000,0.000000,12.655955,Revenue_CL,12.655955
21,57,0,28,157,1,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,0,0,1,0.000000,0.000000,11.327915,Revenue_CL,11.327915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,1516,0,40,192,1,0.0,0.0,0.0,1.0,0.0,...,23.0,10.0,1,0,1,-1.407454,0.000000,12.405535,Revenue_CL,12.405535
629,1569,0,3,152,1,3.0,32.0,0.0,0.0,0.0,...,13.0,6.0,1,0,0,11.143192,0.000000,0.000000,Revenue_MF,11.143192
636,1597,0,19,181,1,0.0,0.0,1.0,0.0,0.0,...,6.0,4.0,0,0,1,0.000000,0.000000,14.286525,Revenue_CL,14.286525
638,1599,0,25,220,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,1,0.000000,0.000000,16.119829,Revenue_CL,16.119829


Selecting top 100 clients based on total revenue:

In [42]:
top_clients.sort_values(by='Max_Revenue', ascending=False).head(100)
# Can then be exported

,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,...,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sale_MF,Sale_CC,Sale_CL,Revenue_MF,Revenue_CC,Revenue_CL,Product_to_offer,Max_Revenue
430,1077,0,26,25,2,1.0,3.0,0.0,0.0,0.0,...,24.0,21.0,1,1,1,18.853127,118.102207,5.023068,Revenue_CC,118.102207
211,506,0,20,78,1,1.0,0.0,1.0,0.0,0.0,...,9.0,11.0,1,0,0,79.394160,0.000000,0.000000,Revenue_MF,79.394160
80,197,0,70,58,1,1.0,4.0,0.0,0.0,0.0,...,0.0,3.0,0,1,0,0.000000,55.977216,0.000000,Revenue_CC,55.977216
308,766,0,32,95,1,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,1,0,0,53.912472,0.000000,0.000000,Revenue_MF,53.912472
608,1510,0,19,44,1,0.0,0.0,1.0,1.0,0.0,...,0.0,12.0,1,0,0,38.803331,0.000000,0.000000,Revenue_MF,38.803331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,701,0,39,151,3,1.0,0.0,0.0,0.0,0.0,...,3.0,1.0,0,1,1,0.000000,4.243516,7.471203,Revenue_CL,7.471203
394,978,0,35,44,1,1.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0,1,0,0.000000,7.186515,0.000000,Revenue_CC,7.186515
518,1289,0,12,20,1,3.0,4.0,0.0,0.0,0.0,...,14.0,8.0,0,1,1,0.000000,-29.563370,7.112948,Revenue_CL,7.112948
499,1241,0,2,170,1,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0,1,0,0.000000,6.996164,0.000000,Revenue_CC,6.996164


Printing number of products to offer:

In [43]:
top_clients['Product_to_offer'].head(100).value_counts().rename_axis('Products').reset_index(name='Count')

,Products,Count
0,Revenue_CL,52
1,Revenue_CC,28
2,Revenue_MF,20


Printing total potential revenue from top 100 clients:

In [44]:
top_clients['Max_Revenue'].sort_values(ascending=False).head(100).sum()

1570.978268085931